# [LVV-T231] - M13T-004: Individual Hardpoint Breakaway Test

Notebook containing data analysis for the [LVV-T231] test case.  
The script used to run this test case can be found in [M13T004.py].  

[LVV-T231]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T231
[M13T004.py]: https://github.com/lsst-ts/ts_m1m3supporttesting/blob/develop/M13T004.py

## Summary

1. The analysis comprining the measured hardpoint stiffness from specified stiffness.
2. This notebook analyses the offline data produces by the script [M13T004.py] and stored on m1m3-dev.cp.lsst.org.
3. The data show the forces changing with hardpoint extension and compression.
4. Compression is marked bya "-" infront of "-999999999"in the file name.

## Expected Results

It is expected that the fitted hardpoint stiffness is within the specified stiffness. 



In [2]:
test_case = "LVV-T231"
test_exec = "LVV-EXXXX" 

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [6]:
# Directory with csv files
directory = '/home/hdrass/'

In [8]:
# Loop through files in directory
for filename in os.listdir(directory):
    if filename.startswith("HP"):
        print(filename)
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        
        zero = 450          # plot only regions around 0 for "Measured force"

        # Limit data to region around zero force if specified
        df_sort = df.iloc[(df[df.columns[2]]).abs().argsort()[:2]]
        zindex = df_sort.index.tolist()[0]
        df = df[max(0, zindex - zero) : zindex + zero]

        scale = 0.2442      # scale factor for "encoder"
        df[df.columns[3]] *= scale

        # Extract x and y data
        x = df[df.columns[3]]
        y = df[df.columns[2]]

        # Find value of x at zero force
        zero_force = df.loc[df[df.columns[2]].abs().idxmin(), df.columns[3]]

        points = 50  # number of points around zero force for linear fit

        # Extract x and y data around zero force
        x_near_zero = x[(x > zero_force - points ) & (x < zero_force + points )]
        y_near_zero = y[(x > zero_force - points ) & (x < zero_force + points )]

        # Calculate slope of linear fit to data around zero force
        m, b = np.polyfit(x_near_zero, y_near_zero, 1)

        x_plot_mean = x[(x > zero_force - 100) & (x < zero_force + 100)]
        x_plot_spec = x[(x > zero_force - 28) & (x < zero_force + 28)]

        spect_stiffness  = 100

        plt.figure(figsize=(15,10))
        df.plot(df.columns[3], df.columns[2])
        plt.plot(
            x_plot_mean,
            m*x_plot_mean + b, color='red', 
            linewidth=2, 
            label=f"Calculated Stiffness = {m:.2f} N/μm\n"
                  f"Encoder Fitting Range =  {round(x_near_zero.min(),2)}" 
                  f" - {round(x_near_zero.max(),2)} μm"
            )
        plt.plot(
            x_plot_spec, 
            spect_stiffness*(x_plot_spec - zero_force), 
            color='green', 
            linewidth=2, 
            label=f"Spect. Stiffness = 100 N/μm"
            )
        legend = plt.legend(loc='upper left', fontsize=8)

        # Add axis labels
        plt.xlabel('Encoder (μm)')
        plt.ylabel('Force (N)')

        # Add grid and legend to plot
        plt.grid(True, linestyle='--', linewidth=0.5)

        # Modify x-axis to have zero in the middle
        ax = plt.gca()
        ax.spines['bottom'].set_position('zero')
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.xaxis.set_ticks_position('bottom')

        # Add title to plot
        plt.title(f"{filename}")
        
        # Save plot as png file
        plt.savefig(f'{directory}/{filename}.png',dpi=300,bbox_inches='tight')

        # Show plot
        plt.show()